In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def create_tridiagonal(y,x,a,b,c,d,**kwargs):
    BC_L = kwargs["Left"]
    BC_R = kwargs["Right"]
    args = kwargs["args"]
    Nx   = len(x)
   
    if BC_L=="Dirichlet" and BC_R=="Dirichlet":
        d[1]    = d[1]    - a[1]*y[0]
        d[Nx-2] = d[Nx-2] - c[Nx-2]*y[Nx-1]
        
    if BC_L=="Dirichlet" and BC_R=="Neumann":
        d[1]    = d[1]    - a[1]*y[0]
        b[Nx-2] = b[Nx-2] + c[Nx-2]
            
    if BC_L=="Neumann" and BC_R=="Dirichlet":
        b[1]    = b[1]    + a[1]
        d[Nx-2] = d[Nx-2] - c[Nx-2]*y[Nx-1]
    
    return a,b,c,d

In [3]:
def ThomasAlgorithm(create_matrix,y,x,**kwargs):
    BC_L = kwargs["Left"]
    BC_R = kwargs["Right"]
    
    a,b,c,d = create_matrix(y,x,**kwargs)
    a,b,c,d = create_tridiagonal(y,x,a,b,c,d,**kwargs)
    
    # Thomas Algorithm:
    # Forward calculation
    Nx = len(x)
    y1 = np.zeros(len(x))
    for n in range(2,Nx-1,1):
        #a[n] = -a[n]/b[n-1]*b[n-1] + a[n]
        b[n] = -a[n]/b[n-1]*c[n-1] + b[n]
        d[n] = -a[n]/b[n-1]*d[n-1] + d[n]
    # Backward substitution
    for n in range(Nx-2,0,-1):
        if n == Nx-2:
            y1[n] = d[n]/b[n]
        else:
            y1[n] = (d[n]-c[n]*y[n+1])/b[n]

    if BC_L=="Dirichlet" and BC_R=="Neumann":
        y1[Nx-1] = y1[Nx-2]
    if BC_L=="Neumann" and BC_R=="Dirichlet":
        y1[0]    = y1[1]
        
    return y1